In [1]:
import secrets
import numpy as np
from sympy import Integer
import cv2
# def generate_random_binary_iv(length):
#     iv = secrets.randbits(length)
#     return bin(iv)[2:].zfill(length)

# # Tạo khóa IV có chiều dài 128 bit
# IV = generate_random_binary_iv(128)

# # In ra khóa IV


# import random

# def generate_random_binary_string(length):
#     return ''.join(str(random.randint(0, 1)) for _ in range(length))

# # Tạo chuỗi nhị phân ngẫu nhiên có độ dài 278 bit
# K = generate_random_binary_string(278)

# # In ra chuỗi nhị phân K

IV= '01001010010011110110010101100101111011001010111011010101010001110101010000010000100001100010011000111001001010111100101011101001'
K= '10111000101001001111110100111010110110011101011110111001011100110110101111011011001111010011011100101101101110001110000100000010100001010100010100101001001000110110100000000111100011000000100111100010001000011011010011010011010000001000101100110111010010100010001110011000011100'
print("Khóa IV:", IV)
print("Chuỗi nhị phân K:", K)
def Xor(bin1, bin2):
    # Convert binary strings to integers
    int1 = int(bin1, 2)
    int2 = int(bin2, 2)
    
    # Perform XOR operation
    result = int1 ^ int2
    
    # Convert result back to binary string
    result_bin = bin(result)[2:].zfill(32)
    
    return result_bin


# Các giá trị ban đầu
IVP = IV[:32]
XP0 = K[:32]
IVS = IV[32:64]
XS0 = K[32:64]
IVL = IV[64:96]
XL0 = K[64:96]
IVT = IV[96:128]
XT0 = K[96:128]
Q0 = K[128:160]
Ps = K[160:192]
Pp = K[192:223]
Tr = K[223:233]
Kd = K[258:278]
# Thực hiện phép XOR cho mỗi biến
result_IVP_XP0 = Xor(IVP, XP0)
result_IVS_XS0 = Xor(IVS, XS0)
result_IVL_XL0 = Xor(IVL, XL0)
result_IVT_XT0 = Xor(IVT, XT0)

# In kết quả
print("IVP ^ XP0:", result_IVP_XP0)
print("IVS ^ XS0:", result_IVS_XS0)
print("IVL ^ XL0:", result_IVL_XL0)
print("IVT ^ XT0:", result_IVT_XT0)
# Các giá trị nhị phân của các phần tử e
e_binary = np.array([[0,K[233:238], K[238:243], K[243:248]],
                     [K[248:253],0, K[253:258], K[258:263]],
                     [K[263:268], K[268:273],0, K[273:278]],
                     [K[233:238], K[238:243], K[243:248],0 ]])

# Chuyển đổi các giá trị nhị phân sang thập phân
e_decimal = np.vectorize(lambda x: int(str(x), 2))(e_binary)

# Các giá trị của các phần tử e ở dạng thập phân
print("Giá trị của các phần tử e ở dạng thập phân:")
print(e_decimal)

# Giá trị mặc định của N
N = 32

# Tính toán các giá trị mới của M11, M22, M33, và M44
M11 = 2 ** N - np.sum(e_decimal, axis=1)[0]
M22 = 2 ** N - np.sum(e_decimal, axis=1)[1]
M33 = 2 ** N - np.sum(e_decimal, axis=1)[2]
M44 = 2 ** N - np.sum(e_decimal, axis=1)[3]

# Tạo ma trận M
new_values = np.array([[M11,0,0,0], 
              [0,M22,0,0], 
              [0,0,M33,0], 
              [0,0,0,M44]])
print("M",new_values)
print("M" ,M11)
def add_matrices(e_decimal, new_values):
    M = [[0]*4 for _ in range(4)]  # Tạo ma trận kết quả với tất cả các phần tử là 0

    # Thực hiện phép cộng từng phần tử của hai ma trận
    for i in range(4):
        for j in range(4):
            M[i][j] = e_decimal[i][j] + new_values[i][j]

    return M
# Thực hiện phép cộng
M = add_matrices(e_decimal, new_values)

# In ma trận kết quả
for row in M:
    print(row)

Khóa IV: 01001010010011110110010101100101111011001010111011010101010001110101010000010000100001100010011000111001001010111100101011101001
Chuỗi nhị phân K: 10111000101001001111110100111010110110011101011110111001011100110110101111011011001111010011011100101101101110001110000100000010100001010100010100101001001000110110100000000111100011000000100111100010001000011011010011010011010000001000101100110111010010100010001110011000011100
IVP ^ XP0: 11110010111010111001100001011111
IVS ^ XS0: 00110101011110010110110000110100
IVL ^ XL0: 00111111110010111011101100010001
IVT ^ XT0: 00010100100100110010101111101011
Giá trị của các phần tử e ở dạng thập phân:
[[ 0  2 25 23]
 [ 9  0  8 17]
 [25 16  0 28]
 [ 2 25 23  0]]
M [[4294967246          0          0          0]
 [         0 4294967262          0          0]
 [         0          0 4294967227          0]
 [         0          0          0 4294967246]]
M 4294967246
[4294967246, 2, 25, 23]
[9, 4294967262, 8, 17]
[25, 16, 4294967227, 28]
[2, 25, 

In [2]:
def calculate_XP(N, Pp_decimal, XP_prev):
    
    if not (1 <= Pp_decimal <= 2**(N-1) - 1) or not (1 <= XP_prev <= 2**N - 1):
        raise ValueError("Pp và XP_prev phải nằm trong khoảng cho phép.")
    
    # Tính toán giá trị XP(1) dựa trên công thức đã cho
    if XP_prev > 0 and XP_prev < Pp_decimal:
        XP_n = (2**N * XP_prev) // Pp_decimal
    elif XP_prev > Pp_decimal and XP_prev < 2**(N-1):
        XP_n = (2**N * (XP_prev - Pp_decimal)) // (2**(N-1) - Pp_decimal)
    elif XP_prev > 2**(N-1) and XP_prev < 2**N - Pp_decimal:
        XP_n = (2**N * (2**N - XP_prev - Pp_decimal)) // (2**(N-1) - Pp_decimal)
    elif XP_prev > 2**N - Pp_decimal and XP_prev < 2**N:
        XP_n = (2**N * (2**N - XP_prev)) // Pp_decimal
    else:
        XP_n = 2**N
    
    return XP_n

# Giả sử các giá trị đã cho
N = 32
# Pp_binary = '01111111111111111111111111111111'  # Pp ở dạng nhị phân 31 bit
Pp_binary = Pp
# XP_prev_binary = '00111111111111111111111111111111'  # XP(0) ở dạng nhị phân 32 bit
XP_prev_binary = result_IVP_XP0
# Chuyển đổi XP(0) từ dạng nhị phân sang thập phân
XP_prev = int(XP_prev_binary, 2) % 2**N
# Chuyển đổi Pp từ dạng nhị phân sang thập phân
Pp_decimal = int(Pp_binary, 2) % 2**(N-1)
# Tính toán giá trị XP(1)
XP_n = calculate_XP(N, Pp_decimal, XP_prev)

print(f"Giá trị XP(1) là: {XP_n}")
############
def calculate_XL( N, XL_prev):
   # XL_prev = XL_prev_mod_N % (2 ** N)
    if XL_prev != (3 * (2 ** (N - 2)) - 1) and XL_prev != (2 ** (N - 1)):
        XL_n = (XL_prev * (2 ** N - XL_prev)// 2**(N-2)) % (2 ** N)
    elif XL_prev == (3 * (2 ** (N - 2)) - 1):
        XL_n = (3 * (2 ** (N - 2))) * (2 ** (N - 1)) % (2 ** N)
    else:
        XL_n = (2 ** (N - 1)) * (2 ** (N - 1)) % (2 ** N)
    return XL_n

# Example usage:
N = 32  # Độ dài của dạng nhị phân
# XL_prev_mod_N ='10000010001001001000010111010001'
XL_prev_mod_N = result_IVL_XL0
# XL_prev_mod_N = int(input("Nhập giá trị XL(n-1) dạng nhị phân 32 bit mod 2**N: "), 2)
XL_prev = int(XL_prev_mod_N, 2) % (2 ** N)
XL_n = calculate_XL( N ,XL_prev)
 
print("Giá trị XL(1) dạng thập phân mod 2**N:", XL_n)
######
def calculate_XS(N, Ps_decimal, XS_prev):
   
   
   
    # Tính toán giá trị XS(1) dựa trên công thức đã cho
    if 0 < XS_prev < Ps_decimal:
        XS_n = (2**N * XS_prev) // Ps_decimal % (2 ** N)
    elif Ps_decimal < XS_prev < 2**N:
        XS_n = (2**N * (2**N - XS_prev)) // (2**N - Ps_decimal) % (2 ** N)
    else:
        XS_n = 2**N - 1 
        
    return int(XS_n)

# Thử nghiệm với giá trị ban đầu 
N = 32
# Ps_binary = "11011000101101101110010110110110"  # Giả sử giá trị nhị phân của Ps
# XS_prev_binary = "10101010101010101010101010101010"  # Giả sử giá trị nhị phân của XS(0)
Ps_binary = Ps
XS_prev_binary = result_IVS_XS0
 # Chuyển đổi XS(0) từ dạng nhị phân sang thập phân
XS_prev = int(XS_prev_binary, 2) % 2**N
    # Chuyển đổi Ps từ dạng nhị phân sang thập phân
Ps_decimal = int(Ps_binary, 2) % 2**N
# Tính toán giá trị XS(1)
XS_n = calculate_XS(N, Ps_decimal, XS_prev)

print(f"Giá trị XS(1) là: {XS_n}")
###########
from sympy import Integer
def calculate_XT(N, XT_prev):
    # Chuyển XT_prev sang kiểu Integer để xử lý với số lớn
    XT_prev = Integer(XT_prev)
    # Tính toán giá trị XS(1) dựa trên công thức đã cho
    XT_n = (2 ** (-2*N + 2) * (4 * (XT_prev - 2**(N-1))**3 - 3 * 2**(2*N-2) * (XT_prev - 2**(N-1))) + 2**(N-1)) % 2**N
    return max(1, min(int(XT_n) % (2**N), 2**N - 1))  # Đảm bảo giá trị nằm trong khoảng từ 1 đến 2^N - 1

# Gán giá trị cho N và XT_prev_binary
N = 32
# result_IVT_XT0 = "10101010101010101010101010101010"  # Giả sử dữ liệu đầu vào là chuỗi nhị phân
XT_prev_binary = result_IVT_XT0
# Chuyển đổi XT(0) từ dạng nhị phân sang thập phân
XT_prev = int(XT_prev_binary, 2) % 2**N
    # Tính toán giá trị XT(1)
XT_n = calculate_XT(N, XT_prev)

print(f"Giá trị XT(1) là: {XT_n}")

# Chuyển đổi và in ra dạng nhị phân 32 bit
XT_n_2 = bin(XT_n)[2:].zfill(32)
print("Giá trị nhị phân 32 bit:", XT_n_2)
class LFSR:
    
    def __init__(self, seed, polynomial):
        self.state = seed
        self.polynomial = polynomial

    def shift(self):
        #Tính toán giá trị
        feedback = (self.state >> 32) ^ ((self.state >> 22) & 1) ^ ((self.state >> 3) & 1) ^ ((self.state >> 2) & 1)
        # Dịch chuyển trạng thái
        self.state = (self.state >> 1) | (feedback << 31)

        
    def get_output(self):
        return self.state
seed_binary = Q0
seed_decimal = int(seed_binary, 2)
# Xác định đa thức nguyên thủy
primitive_polynomial = [32, 22, 2, 1, 0]

# Khởi tạo LFSR với giá trị khởi đầu và đa thức
lfsr = LFSR(seed=seed_decimal, polynomial=primitive_polynomial)

# Tạo ra 10 đầu ra
for _ in range(13):
    print(bin(lfsr.get_output())[2:].zfill(32))
    lfsr.shift()
###################
# Khởi tạo LFSR với giá trị seed và đa thức
lfsr = LFSR(seed=seed_decimal, polynomial=primitive_polynomial)

# Lặp 14 lần để dịch chuyển LFSR
for _ in range(54):
    lfsr.shift()

# Lấy giá trị của LFSR tại thời điểm hiện tại (lần thứ 14)
LFSR_n = lfsr.get_output()

# In ra giá trị hiện tại của LFSR
print (LFSR_n)
print(bin(LFSR_n)[2:].zfill(32))
#Tính XTI_1 

    # Chuyển đổi XT(0) từ dạng nhị phân sang thập phân
XTI_n_2 = Xor(XT_n_2,bin(LFSR_n)[2:].zfill(32))
XTI_n = int(XTI_n_2, 2) % 2**N
  


# In ra giá trị XTI_n dạng thập phân
print("XTI_n:", XTI_n)

Giá trị XP(1) là: 496851341
Giá trị XL(1) dạng thập phân mod 2**N: 3214363505
Giá trị XS(1) là: 2207743419
Giá trị XT(1) là: 2476545581
Giá trị nhị phân 32 bit: 10010011100111010001011000101101
10000101010001010010100100100011
11000010101000101001010010010001
01100001010100010100101001001000
00110000101010001010010100100100
10011000010101000101001010010010
11001100001010100010100101001001
11100110000101010001010010100100
11110011000010101000101001010010
01111001100001010100010100101001
10111100110000101010001010010100
01011110011000010101000101001010
00101111001100001010100010100101
10010111100110000101010001010010
2372917383
10001101011011111101100010000111
XTI_n: 519229098


In [3]:
import numpy as np
# Tạo ma trận 4x1 và đưa giá trị XP_n vào
matrix_4x1 = np.zeros((4, 1))
matrix_4x1[0, 0] = XP_n
matrix_4x1[1, 0] = XS_n
matrix_4x1[2, 0] = XL_n
matrix_4x1[3, 0] = XTI_n
print("Ma trận 4x1:")
print(matrix_4x1)

Ma trận 4x1:
[[4.96851341e+08]
 [2.20774342e+09]
 [3.21436350e+09]
 [5.19229098e+08]]


In [4]:

# Ma trận 4x1
matrix_4x1 = np.array([[
    XP_n], [XS_n], [XL_n], [XTI_n]])

# Ma trận M

# Nhân hai ma trận
M_A = (M @ matrix_4x1) % (2**32)

print("Kết quả sau khi nhân ma trận:")
print(M_A)
import numpy as np

# Giả sử result_matrix đã được tính trước đó

# Lấy kết quả của từng hàng
row1_result = M_A[0, 0]
row2_result = M_A[1, 0]
row3_result = M_A[2, 0]
row4_result = M_A[3, 0]

# Lấy phần dư mod 2^32
XPC_1 = M_A[0, 0] % (2**32)
XSC_1 = M_A[1, 0] % (2**32)
XLC_1 = M_A[2, 0] % (2**32)
XTIC_1 = M_A[3, 0] % (2**32)

print("Kết quả của từng hàng sau khi lấy phần dư mod 2^32:")
print("Row 1:", XPC_1)
print("Row 2:", XSC_1)
print("Row 3:", XLC_1)
print("Row 4:", XTIC_1)
########Tính toán Xn
Xn = XPC_1 ^ XSC_1 ^ XLC_1 ^ XTIC_1 
print(Xn)
# Lưu trữ giá trị ban đầu
XP_prev = XPC_1
XS_prev = XSC_1
XL_prev = XLC_1
XT_prev = XTIC_1
LFSR_prev = LFSR_n
Xn_values = []

Kết quả sau khi nhân ma trận:
[[3154799931]
 [2604894193]
 [3701268376]
 [1076978768]]
Kết quả của từng hàng sau khi lấy phần dư mod 2^32:
Row 1: 3154799931
Row 2: 2604894193
Row 3: 3701268376
Row 4: 1076978768
3152302338


In [5]:

# Tính toán các giá trị XP(n), XL(n), XS(n), XT(n) với n trong khoảng 2 < n < 100
for n in range(2, 1600):
    Xn_values.append(Xn)
    # Tính toán giá trị XP, XL, XS, XT cho vòng lặp hiện tại
    XP_n = calculate_XP(N, Pp_decimal, XP_prev)
    XL_n = calculate_XL(N, XL_prev)
    XS_n = calculate_XS(N, Ps_decimal, XS_prev)
    XT_n = calculate_XT(N, XT_prev)
    
    # Cập nhật giá trị LFSR
    lfsr.shift()
    LFSR_n = lfsr.get_output()
    
    # Tính giá trị XT_n
    XTI_n = (XT_n ^ LFSR_n) % 2**N
    
    
    # In ra giá trị của XP(n), XL(n), XS(n), XT(n)
    # print(f"Giá trị XP({n}) là: {XP_n}")
    # print(f"Giá trị XL({n}) là: {XL_n}")
    # print(f"Giá trị XS({n}) là: {XS_n}")
    # print(f"Giá trị XT({n}) là: {XT_n}")

    # Tính toán Xn và in ra giá trị cuối cùng
    matrix_4x1 = np.zeros((4, 1))
    matrix_4x1[0, 0] = XP_n
    matrix_4x1[1, 0] = XS_n
    matrix_4x1[2, 0] = XL_n
    matrix_4x1[3, 0] = XTI_n

    matrix_4x1 = np.array([[XP_n], [XS_n], [XL_n], [XTI_n]])
    # Ma trận M
    # Nhân hai ma trận
    M_A = (M @ matrix_4x1) % (2**32)

    XPC_n = M_A[0, 0] % (2**32)
    XSC_n = M_A[1, 0] % (2**32)
    XLC_n = M_A[2, 0] % (2**32)
    XTIC_n = M_A[3, 0] % (2**32)
    
    # Cập nhật giá trị cho vòng lặp tiếp theo
    XP_prev = XPC_n
    XS_prev = XSC_n
    XL_prev = XLC_n
    XT_prev = XTIC_n
    LFSR_prev = LFSR_n
    # Tính toán Xn và in ra giá trị cuối cùng
    Xn = XPC_n ^ XSC_n ^ XLC_n ^ XTIC_n 
    # print(f"Giá trị Xn({n}) là: {Xn}")
    ####################
  

In [6]:
# Khởi tạo một set để lưu trữ các giá trị đã xuất hiện
S_values = set()

# Khởi tạo S_box 16x16 với tất cả các giá trị ban đầu là 0
S_box = [[0 for _ in range(16)] for _ in range(16)]

# Biến đếm số lượng giá trị duy nhất đã được thêm vào S_box
t = 0

# Tính toán V = Xn mod 256 cho mỗi giá trị Xn và đẩy vào S_box
for Xn in Xn_values:
    V = Xn % 256
    # Nếu giá trị V chưa xuất hiện, thêm vào S_box và vào set unique_values
    if V not in S_values:
        # Tìm vị trí cho giá trị V trong S_box
        row = t // 16
        col = t % 16
        # Lưu giá trị V vào S_box tại vị trí tương ứng
        S_box[row][col] = V
        # Thêm giá trị V vào set unique_values
        S_values.add(V)
        # Tăng biến đếm lên 1
        t += 1
    # Nếu S_box đã đầy 256 giá trị, dừng vòng lặp
    if t > 255:
        break

# In ra S_box
print("S_box 16x16:")
for row in S_box:
    print(row)


S_box 16x16:
[2, 193, 0, 32, 162, 160, 134, 165, 41, 16, 228, 252, 169, 254, 38, 113]
[182, 213, 44, 237, 92, 119, 184, 240, 216, 147, 4, 233, 129, 192, 244, 197]
[176, 109, 122, 39, 224, 247, 49, 30, 51, 98, 102, 110, 211, 231, 20, 12]
[7, 205, 27, 179, 57, 73, 56, 151, 126, 105, 141, 104, 242, 203, 171, 236]
[8, 223, 209, 65, 25, 83, 74, 234, 106, 186, 204, 207, 153, 11, 81, 45]
[71, 180, 154, 232, 101, 97, 133, 225, 181, 99, 103, 17, 125, 120, 155, 86]
[59, 227, 31, 9, 175, 189, 158, 15, 24, 200, 95, 94, 1, 90, 123, 245]
[19, 76, 167, 144, 127, 210, 132, 22, 202, 69, 77, 156, 251, 201, 79, 115]
[87, 61, 82, 212, 37, 195, 198, 62, 85, 128, 214, 93, 29, 117, 107, 239]
[42, 116, 206, 222, 137, 183, 88, 148, 46, 150, 5, 174, 67, 188, 136, 84]
[89, 241, 18, 3, 108, 191, 143, 112, 138, 52, 163, 63, 118, 78, 235, 172]
[53, 173, 190, 159, 91, 140, 246, 168, 238, 217, 166, 187, 13, 170, 142, 199]
[68, 28, 75, 100, 10, 243, 145, 54, 23, 194, 215, 220, 111, 80, 219, 161]
[250, 114, 149, 14, 34

In [7]:
# Khởi tạo danh sách mới
sbox = []
# Duyệt qua từng hàng của ma trận và thêm các phần tử vào danh sách mới
for row in S_box:
    sbox.extend(row)
# In danh sách đã được làm phẳng
print(sbox)
# Tạo S-box ngược lại
sbox_inv = np.argsort(sbox)
print(sbox_inv)



            

[2, 193, 0, 32, 162, 160, 134, 165, 41, 16, 228, 252, 169, 254, 38, 113, 182, 213, 44, 237, 92, 119, 184, 240, 216, 147, 4, 233, 129, 192, 244, 197, 176, 109, 122, 39, 224, 247, 49, 30, 51, 98, 102, 110, 211, 231, 20, 12, 7, 205, 27, 179, 57, 73, 56, 151, 126, 105, 141, 104, 242, 203, 171, 236, 8, 223, 209, 65, 25, 83, 74, 234, 106, 186, 204, 207, 153, 11, 81, 45, 71, 180, 154, 232, 101, 97, 133, 225, 181, 99, 103, 17, 125, 120, 155, 86, 59, 227, 31, 9, 175, 189, 158, 15, 24, 200, 95, 94, 1, 90, 123, 245, 19, 76, 167, 144, 127, 210, 132, 22, 202, 69, 77, 156, 251, 201, 79, 115, 87, 61, 82, 212, 37, 195, 198, 62, 85, 128, 214, 93, 29, 117, 107, 239, 42, 116, 206, 222, 137, 183, 88, 148, 46, 150, 5, 174, 67, 188, 136, 84, 89, 241, 18, 3, 108, 191, 143, 112, 138, 52, 163, 63, 118, 78, 235, 172, 53, 173, 190, 159, 91, 140, 246, 168, 238, 217, 166, 187, 13, 170, 142, 199, 68, 28, 75, 100, 10, 243, 145, 54, 23, 194, 215, 220, 111, 80, 219, 161, 250, 114, 149, 14, 34, 33, 66, 26, 185, 72, 48,

In [8]:
def delete_images_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isdir(file_path):
            delete_images_in_folder(file_path)  # Đệ quy để xóa hình ảnh trong thư mục con
        elif filename.endswith(".png") or filename.endswith(".jpg"):  # Kiểm tra đuôi file
            os.remove(file_path)

In [9]:
AES_Sbox = [
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16
]
AES_Sbox_inv = [
    0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,
    0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,
    0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,
    0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,
    0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,
    0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,
    0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,
    0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,
    0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,
    0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,
    0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
    0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,
    0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,
    0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,
    0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d,
]
# sbox_hex = [hex(num) for num in sbox]
# sbox_inv_hex = [hex(num) for num in sbox_inv]
# Sbox_aes = [AES_Sbox[int(num, 16)] for num in sbox_hex]
# print(Sbox_aes)
# Sbox_aes_hex = [hex(num) for num in Sbox_aes]
# print(Sbox_aes_hex)
# Sbox_aes_inv = [AES_Sbox_inv[int(num, 16)] for num in sbox_inv_hex]
# print(Sbox_aes_inv)
# Sbox_aes_inv_hex = [hex(num) for num in Sbox_aes_inv]
# print(Sbox_aes_inv_hex)

In [10]:
import cv2


# Đọc hình ảnh
image = cv2.imread('plain_images/lenna512x512.png')

# Chia thành 3 mặt phẳng màu R, G, B
b, g, r = cv2.split(image)

# Tạo các mặt phẳng màu từ các kênh màu
blue_plane = cv2.merge((b, np.zeros_like(g), np.zeros_like(r)))
green_plane = cv2.merge((np.zeros_like(b), g, np.zeros_like(r)))
red_plane = cv2.merge((np.zeros_like(b), np.zeros_like(g), r))

# Hiển thị từng mặt phẳng màu
cv2.imshow('Blue Plane', blue_plane)
cv2.imshow('Green Plane', green_plane)
cv2.imshow('Red Plane', red_plane)

# # Chờ người dùng nhấn một phím bất kỳ để đóng cửa sổ
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# Hàm mã hóa khối 32x32 pixels
def encrypt_block(block):
    for i in range(32):
        for j in range(32):
            block[i, j] = AES_Sbox[sbox[block[i, j]]]
    return block

# Chia mỗi mặt phẳng màu thành các khối 32x32 pixels
blocks_b = [b[i:i+32, j:j+32] for i in range(0, b.shape[0], 32) for j in range(0, b.shape[1], 32)]
blocks_g = [g[i:i+32, j:j+32] for i in range(0, g.shape[0], 32) for j in range(0, g.shape[1], 32)]
blocks_r = [r[i:i+32, j:j+32] for i in range(0, r.shape[0], 32) for j in range(0, r.shape[1], 32)]

# Mã hóa từng khối màu
encrypted_blocks_b = np.array([encrypt_block(block) for block in blocks_b])
encrypted_blocks_g = np.array([encrypt_block(block) for block in blocks_g])
encrypted_blocks_r = np.array([encrypt_block(block) for block in blocks_r])

# Gộp các khối đã mã hóa lại với nhau
encrypted_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(encrypted_blocks_b, b.shape[0]//32)])
encrypted_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(encrypted_blocks_g, g.shape[0]//32)])
encrypted_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(encrypted_blocks_r, r.shape[0]//32)])

# Tạo hình ảnh đã mã hóa
encrypted_image = cv2.merge((encrypted_b, encrypted_g, encrypted_r))

# Hiển thị hình ảnh đã mã hóa
cv2.imshow('Encrypted Image', encrypted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [11]:
# Hàm giải mã khối 32x32 pixels
def decrypt_block(block):
    for i in range(32):
        for j in range(32):
            # Tìm vị trí của giá trị trong Sbox_AES để lấy lại giá trị gốc
            block[i, j] = sbox_inv[AES_Sbox_inv[block[i, j]]]
    return block

# Chia mỗi mặt phẳng màu của hình ảnh đã mã hóa thành các khối 32x32 pixels
encrypted_b_blocks = [encrypted_b[i:i+32, j:j+32] for i in range(0, encrypted_b.shape[0], 32) for j in range(0, encrypted_b.shape[1], 32)]
encrypted_g_blocks = [encrypted_g[i:i+32, j:j+32] for i in range(0, encrypted_g.shape[0], 32) for j in range(0, encrypted_g.shape[1], 32)]
encrypted_r_blocks = [encrypted_r[i:i+32, j:j+32] for i in range(0, encrypted_r.shape[0], 32) for j in range(0, encrypted_r.shape[1], 32)]

# Giải mã từng khối màu
decrypted_blocks_b = np.array([decrypt_block(block) for block in encrypted_b_blocks])
decrypted_blocks_g = np.array([decrypt_block(block) for block in encrypted_g_blocks])
decrypted_blocks_r = np.array([decrypt_block(block) for block in encrypted_r_blocks])

# Gộp các khối đã giải mã lại với nhau
decrypted_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_b, encrypted_b.shape[0]//32)])
decrypted_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_g, encrypted_g.shape[0]//32)])
decrypted_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_r, encrypted_r.shape[0]//32)])

# Tạo hình ảnh đã giải mã
decrypted_image = cv2.merge((decrypted_b, decrypted_g, decrypted_r))

# Hiển thị hình ảnh đã giải mã
cv2.imshow('Decrypted Image', decrypted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [19]:
def modified_2d_cat_map(i, j, ud, vd, rid, rjd, M):
    Z1 = (rid + rjd)
    Z2 = (ud * vd + 1) 
    Z3 = (vd * i + rjd) 
    in_ = (i + ud * j + Z1) % M
    jn = (Z3 + Z2 * j) % M 
    return in_, jn

# Hàm hoán vị vị trí của các pixel trong một khối
def permute_block(block, Kd):
    ud = int(Kd[:5], 2)
    vd = int(Kd[5:10], 2)
    rid = int(Kd[10:15], 2)
    rjd = int(Kd[15:20], 2)
    M = 32
    for i in range(M):
        for j in range(M):
            in_, jn = modified_2d_cat_map(i, j, ud, vd, rid, rjd, M)
            block[in_, jn] = block[i, j]  # Ghi đè giá trị của pixel vào vị trí mới
    return block

# Hoán vị các khối màu
permuted_blocks_b = np.array([permute_block(block, Kd) for block in encrypted_blocks_b])
permuted_blocks_g = np.array([permute_block(block, Kd) for block in encrypted_blocks_g])
permuted_blocks_r = np.array([permute_block(block, Kd) for block in encrypted_blocks_r])

# Gộp các khối đã hoán vị lại với nhau
permuted_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(permuted_blocks_b, encrypted_b.shape[0]//32)])
permuted_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(permuted_blocks_g, encrypted_g.shape[0]//32)])
permuted_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(permuted_blocks_r, encrypted_r.shape[0]//32)])

# Tạo hình ảnh đã hoán vị
permuted_image = cv2.merge((permuted_b, permuted_g, permuted_r))
print(permuted_b)

# Hiển thị hình ảnh đã hoán vị
cv2.imshow('Permuted Image', permuted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



[[110 232  17 ...  17 239 110]
 [122 232 232 ... 160  39 251]
 [ 41 110 122 ... 251 251 243]
 ...
 [  2 157 188 ...   2 137  48]
 [137 110 161 ...   2 190 137]
 [157   8 160 ... 188 137 137]]


In [13]:
# def inverse_modified_2d_cat_map(in_, jn, ud, vd, rid, rjd, M):
#     Z1 = (rid + rjd)
#     Z2 = (ud * vd + 1)
#     Z3 = (vd * in_ - rjd) 
#     i = (in_ - ud * jn - Z1) % M
#     j = (Z3 - Z2 * jn) % M 
#     return i, j

# def inverse_permute_block(permuted_block, Kd):
#     M = 32
#     ud = int(Kd[:5], 2)
#     vd = int(Kd[5:10], 2)
#     rid = int(Kd[10:15], 2)
#     rjd = int(Kd[15:20], 2)
#     for i in range(M):
#         for j in range(M):
#             in_, jn = inverse_modified_2d_cat_map(i, j, ud, vd, rid, rjd, M)
#             permuted_block[i, j] = permuted_block[in_, jn]
#     return permuted_block

# # Áp dụng hàm ngược cho các khối màu đã hoán vị
# inv_permuted_blocks_b = np.array([inverse_permute_block(block, Kd) for block in permuted_blocks_b])
# inv_permuted_blocks_g = np.array([inverse_permute_block(block, Kd) for block in permuted_blocks_g])
# inv_permuted_blocks_r = np.array([inverse_permute_block(block, Kd) for block in permuted_blocks_r])

# # Gộp các khối đã hoán vị ngược lại với nhau
# inv_permuted_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(inv_permuted_blocks_b, permuted_b.shape[0]//32)])
# inv_permuted_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(inv_permuted_blocks_g, permuted_g.shape[0]//32)])
# inv_permuted_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(inv_permuted_blocks_r, permuted_r.shape[0]//32)])

# # Tạo hình ảnh đã hoán vị ngược lại
# inv_permuted_image = cv2.merge((inv_permuted_b, inv_permuted_g, inv_permuted_r))

# # Hiển thị hình ảnh đã hoán vị ngược lại
# cv2.imshow('Inverse Permuted Image', inv_permuted_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# # Giải mã các khối màu đã hoán vị ngược lại
# decrypted_blocks_b = np.array([decrypt_block(block) for block in inv_permuted_blocks_b])
# decrypted_blocks_g = np.array([decrypt_block(block) for block in inv_permuted_blocks_g])
# decrypted_blocks_r = np.array([decrypt_block(block) for block in inv_permuted_blocks_r])

# # Gộp các khối đã giải mã lại với nhau
# decrypted_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_b, encrypted_b.shape[0]//32)])
# decrypted_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_g, encrypted_g.shape[0]//32)])
# decrypted_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_r, encrypted_r.shape[0]//32)])

# # Tạo hình ảnh đã giải mã
# decrypted_image = cv2.merge((decrypted_b, decrypted_g, decrypted_r))

# # Hiển thị hình ảnh đã giải mã
# cv2.imshow('Decrypted Image', decrypted_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [14]:
def inverse_modified_2d_cat_map(in_, jn, ud, vd, rid, rjd, M):
    Z1 = (rid + rjd)
    Z2 = (ud * vd + 1)
    Z3 = (vd * in_ - rjd)   # Đảo ngược phép toán Z3
    i = (in_ + ud * jn + Z1) % M
    j = (Z3 + Z2 * jn) % M 
    return i, j

def inverse_permute_block(permuted_block):
    M = 32
    inv_permuted_block = np.zeros_like(permuted_block)
    ud = int(Kd[:5], 2)
    vd = int(Kd[5:10], 2)
    rid = int(Kd[10:15], 2)
    rjd = int(Kd[15:20], 2)
    for i in range(M):
        for j in range(M):
            in_, jn = inverse_modified_2d_cat_map(i, j, ud, vd, rid, rjd, M)
            inv_permuted_block[i, j] = permuted_block[in_, jn]
    return inv_permuted_block

# Áp dụng hàm ngược cho các khối màu đã hoán vị
inv_permuted_blocks_b = np.array([inverse_permute_block(block) for block in permuted_blocks_b])
inv_permuted_blocks_g = np.array([inverse_permute_block(block) for block in permuted_blocks_g])
inv_permuted_blocks_r = np.array([inverse_permute_block(block) for block in permuted_blocks_r])

# Gộp các khối đã hoán vị ngược lại với nhau
inv_permuted_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(inv_permuted_blocks_b, permuted_b.shape[0]//32)])
inv_permuted_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(inv_permuted_blocks_g, permuted_g.shape[0]//32)])
inv_permuted_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(inv_permuted_blocks_r, permuted_r.shape[0]//32)])
# # Gộp các khối đã hoán vị ngược lại với nhau
# inv_permuted_b = np.vstack([np.hstack(row_blocks) for row_blocks in inv_permuted_blocks_b])
# inv_permuted_g = np.vstack([np.hstack(row_blocks) for row_blocks in inv_permuted_blocks_g])
# inv_permuted_r = np.vstack([np.hstack(row_blocks) for row_blocks in inv_permuted_blocks_r])

# Tạo hình ảnh đã hoán vị ngược lại
inv_permuted_image = cv2.merge((inv_permuted_b, inv_permuted_g, inv_permuted_r))

# Hiển thị hình ảnh đã hoán vị ngược lại
cv2.imshow('Inverse Permuted Image', inv_permuted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Giải mã các khối màu đã hoán vị ngược lại
decrypted_blocks_b = np.array([decrypt_block(block) for block in inv_permuted_blocks_b])
decrypted_blocks_g = np.array([decrypt_block(block) for block in inv_permuted_blocks_g])
decrypted_blocks_r = np.array([decrypt_block(block) for block in inv_permuted_blocks_r])

# # Gộp các khối đã giải mã lại với nhau
decrypted_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_b, encrypted_b.shape[0]//32)])
decrypted_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_g, encrypted_g.shape[0]//32)])
decrypted_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(decrypted_blocks_r, encrypted_r.shape[0]//32)])
# # Gộp các khối đã giải mã lại với nhau
# decrypted_b = np.vstack([np.hstack(row_blocks) for row_blocks in decrypted_blocks_b])
# decrypted_g = np.vstack([np.hstack(row_blocks) for row_blocks in decrypted_blocks_g])
# decrypted_r = np.vstack([np.hstack(row_blocks) for row_blocks in decrypted_blocks_r])

# Tạo hình ảnh đã giải mã
decrypted_image = cv2.merge((decrypted_b, decrypted_g, decrypted_r))

# Hiển thị hình ảnh đã giải mã
cv2.imshow('Decrypted Image', decrypted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
import numpy as np

def HAD_and_VAD_for_block(block, M):
    sph = np.empty((M, M), dtype=np.uint16)
    cl = np.empty((M, M), dtype=np.uint16)

    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:
                sph[i][j] = (block[i][j] + block[M-1][M-1]) % 256
            elif 0 < i <= M-1 and j == 0:
                sph[i][j] = (block[i][j] + block[i-1][M-1]) % 256
            elif 0 <= i <= M-1 and 0 < j <= M-1:
                sph[i][j] = (block[i][j] + block[i][j-1]) % 256
    
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:
                cl[i][j] = (sph[i][j] + sph[M-1][M-1]) % 256
            elif 0 < j <= M-1 and i == 0:
                cl[i][j] = (sph[i][j] + sph[i-1][M-1]) % 256
            elif 0 <= j <= M-1 and 0 < i <= M-1:
                cl[i][j] = (sph[i][j] + sph[i][j-1]) % 256
    
    return sph, cl


permuted_blocks_b = np.array(permuted_blocks_b)
permuted_blocks_g = np.array(permuted_blocks_g)
permuted_blocks_r = np.array(permuted_blocks_r)

# Áp dụng HAD và VAD cho decrypted_b, decrypted_g, và decrypted_r
sph_b_list = []
cl_b_list = []
sph_g_list = []
cl_g_list = []
sph_r_list = []
cl_r_list = []
M=32
for block in permuted_blocks_b:
    sph_b, cl_b = HAD_and_VAD_for_block(block, M)
    sph_b_list.append(sph_b)
    cl_b_list.append(cl_b)

for block in permuted_blocks_g:
    sph_g, cl_g = HAD_and_VAD_for_block(block, M)
    sph_g_list.append(sph_g)
    cl_g_list.append(cl_g)

for block in permuted_blocks_r:
    sph_r, cl_r = HAD_and_VAD_for_block(block, M)
    sph_r_list.append(sph_r)
    cl_r_list.append(cl_r)

import os  # Thêm dòng này lên đầu mã của bạn

# Lưu hình ảnh đã hoán vị ngược lại vào thư mục
output_directory = "HAD_VAD"
output_image_path = os.path.join(output_directory, "inv_permuted_image.jpg")

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

cv2.imwrite(output_image_path, inv_permuted_image)
print("Hình ảnh đã hoán vị ngược lại đã được lưu vào:", output_image_path)


C:\Users\Admin\AppData\Local\Temp\ipykernel_26160\202084215.py:10: RuntimeWarning: overflow encountered in scalar add
  sph[i][j] = (block[i][j] + block[M-1][M-1]) % 256
C:\Users\Admin\AppData\Local\Temp\ipykernel_26160\202084215.py:14: RuntimeWarning: overflow encountered in scalar add
  sph[i][j] = (block[i][j] + block[i][j-1]) % 256
C:\Users\Admin\AppData\Local\Temp\ipykernel_26160\202084215.py:12: RuntimeWarning: overflow encountered in scalar add
  sph[i][j] = (block[i][j] + block[i-1][M-1]) % 256


Hình ảnh đã hoán vị ngược lại đã được lưu vào: HAD_VAD\inv_permuted_image.jpg


In [16]:
M=32
def HAD(spl, M):
    # Tạo một ma trận kích thước MxM để lưu kết quả
    sph = [[0 for _ in range(M)] for _ in range(M)]
    
    # Tính toán giá trị cho từng phần tử trong ma trận
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:  # Trường hợp khi i = j = 1
                sph[i][j] = spl[i][j] + (spl[M-1][M-1]) % 256 
            elif 0 < i <= M-1 and j == 0:  # Trường hợp khi 1 < i ≤ M và j = 1
                sph[i][j] = spl[i][j] + (spl[i-1][M-1]) % 256
            elif 0 <= i <= M-1 and 0 < j <= M-1:  # Trường hợp chung
                sph[i][j] = spl[i][j] + (spl[i][j-1]) % 256
        
    return sph
def VAD(sph, M):
    # Tạo một ma trận kết quả mới kích thước MxM
    cl = [[0 for _ in range(M)] for _ in range(M)]

    # Tính toán giá trị cho từng phần tử trong ma trận cl
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:  # Trường hợp khi i = j = 1
                cl[i][j] = sph[i][j] + (sph[M-1][M-1]) % 256 
            elif 0 < j <= M-1 and i == 0:  # Trường hợp khi 1 < i ≤ M và j = 1
                cl[i][j] = sph[i][j] + (sph[i-1][M-1]) % 256
            elif 0 <= j <= M-1 and 0 < i <= M-1:  # Trường hợp chung
                cl[i][j] = sph[i][j] + (sph[i][j-1]) % 256
    
    return cl
# Áp dụng HAD cho mỗi khối màu
had_blocks_b = np.array([HAD(block, M) for block in permuted_blocks_b])
had_blocks_g = np.array([HAD(block, M) for block in permuted_blocks_b])
had_blocks_r = np.array([HAD(block, M) for block in permuted_blocks_b])

# Áp dụng VAD cho mỗi khối đã qua HAD
vad_blocks_b = np.array([VAD(block, M) for block in had_blocks_b])
vad_blocks_g = np.array([VAD(block, M) for block in had_blocks_g])
vad_blocks_r = np.array([VAD(block, M) for block in had_blocks_r])

# Gộp các khối đã qua VAD lại với nhau
vad_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(vad_blocks_b, permuted_b.shape[0]//M)])
vad_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(vad_blocks_g, permuted_g.shape[0]//M)])
vad_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(vad_blocks_r, permuted_r.shape[0]//M)])

# Tạo hình ảnh từ kết quả VAD
vad_image = cv2.merge((vad_b, vad_g, vad_r))

import os

# Tạo thư mục nếu nó chưa tồn tại
output_folder = 'HAD_VAD'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Lưu hình ảnh đã qua xử lý vào thư mục
output_path = os.path.join(output_folder, 'vad_image.jpg')
cv2.imwrite(output_path, vad_image)




True

In [17]:
def HAD(spl, M):
    # Tạo một ma trận kích thước MxM để lưu kết quả
    sph = [[0 for _ in range(M)] for _ in range(M)]

    # Tính toán giá trị cho từng phần tử trong ma trận
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:  # Trường hợp khi i = j = 1
                sph[i][j] = spl[i][j] + spl[M-1][M-1] % 256 
            elif 0 < i <= M-1 and j == 0:  # Trường hợp khi 1 < i ≤ M và j = 1
                sph[i][j] = spl[i][j] + spl[i-1][M-1] % 256
            elif 0 <= i <= M-1 and 0 < j <= M-1:  # Trường hợp chung
                sph[i][j] = spl[i][j] + spl[i][j-1] % 256
        
    return sph
def VAD(sph, M):
    # Tạo một ma trận kết quả mới kích thước MxM
    cl = [[0 for _ in range(M)] for _ in range(M)]

    # Tính toán giá trị cho từng phần tử trong ma trận cl
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:  # Trường hợp khi i = j = 1
                cl[i][j] = sph[i][j] + sph[M-1][M-1] % 256 
            elif 0 < j <= M-1 and i == 0:  # Trường hợp khi 1 < i ≤ M và j = 1
                cl[i][j] = sph[i][j] + sph[i-1][M-1] % 256
            elif 0 <= j <= M-1 and 0 < i <= M-1:  # Trường hợp chung
                cl[i][j] = sph[i][j] + sph[i][j-1] % 256
    
    return cl

# Kích thước của mỗi khối ảnh
M = 32

diffused_blocks_b_sph = np.array([HAD(permute_block(block), M) for block in permuted_blocks_b])
diffused_blocks_b_cl = np.array([VAD(sph, M) for sph in diffused_blocks_b_sph])

diffused_blocks_g_sph = np.array([HAD(permute_block(block), M) for block in permuted_blocks_g])
diffused_blocks_g_cl = np.array([VAD(sph, M) for sph in diffused_blocks_g_sph])

diffused_blocks_r_sph = np.array([HAD(permute_block(block), M) for block in permuted_blocks_r])
diffused_blocks_r_cl = np.array([VAD(sph, M) for sph in diffused_blocks_r_sph])

# # Gộp các khối đã khuếch tán lại với nhau
# diffused_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(diffused_blocks_b_cl, permuted_b.shape[0]//M)])
# diffused_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(diffused_blocks_g_cl, permuted_g.shape[0]//M)])
# diffused_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(diffused_blocks_r_cl, permuted_r.shape[0]//M)])
# Gộp các khối đã khuếch tán lại với nhau
diffused_b = np.vstack([np.hstack(row_blocks) for row_blocks in diffused_blocks_b_cl])
diffused_g = np.vstack([np.hstack(row_blocks) for row_blocks in diffused_blocks_g_cl])
diffused_r = np.vstack([np.hstack(row_blocks) for row_blocks in diffused_blocks_r_cl])
# Tạo hình ảnh đã khuếch tán
diffused_image = cv2.merge((diffused_b, diffused_g, diffused_r))

import os

# Đường dẫn của thư mục lưu trữ hình ảnh đã khuếch tán
output_directory = "HAD_VAD"

# Tạo thư mục nếu nó chưa tồn tại
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Tạo đường dẫn đầy đủ cho hình ảnh đã khuếch tán
output_image_path = os.path.join(output_directory, "diffused_image.jpg")

# Lưu hình ảnh đã khuếch tán vào thư mục
cv2.imwrite(output_image_path, diffused_image)

# In thông báo khi hoàn tất
print("Hình ảnh đã khuếch tán đã được lưu vào:", output_image_path)




TypeError: permute_block() missing 1 required positional argument: 'Kd'

In [ ]:
def InvVAD(cl, M):
    # Tạo một ma trận kết quả mới kích thước MxM
    sphl = [[0 for _ in range(M)] for _ in range(M)]

    # Tính toán giá trị cho từng phần tử trong ma trận sphl
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:  # Trường hợp khi i = j = 1
                sphl[i][j] = cl[i][j] - (cl[M-1][M-1]) % 256
            elif 0 < i <= M-1 and 0 < j <= M-1:  # Trường hợp khi 1 < i ≤ M và 1 < j ≤ M
                sphl[i][j] = cl[i][j] - (cl[i-1][j]) % 256
            elif i == 0 and 0 < j <= M-1:  # Trường hợp khi i = 1 và 1 < j ≤ M
                sphl[i][j] = cl[i][j] - (cl[M-1][j-1]) % 256
    
    return sphl

def InvHAD(sphl, M):
    # Tạo một ma trận kết quả mới kích thước MxM
    spl = [[0 for _ in range(M)] for _ in range(M)]

    # Tính toán giá trị cho từng phần tử trong ma trận spl
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:  # Trường hợp khi i = j = 1
                spl[i][j] = sphl[i][j] - (sphl[M-1][M-1]) % 256
            elif 0 < i <= M-1 and j == 0:  # Trường hợp khi 1 < i ≤ M và j = 1
                spl[i][j] = sphl[i][j] - (sphl[i-1][M-1]) % 256
            elif 0 <= i <= M-1 and 0 < j <= M-1:  # Trường hợp chung
                spl[i][j] = sphl[i][j] - (sphl[i][j-1]) % 256
        
    return spl


# Khởi tạo kích thước của mỗi khối ảnh
M = 32

# Thực hiện ngược lại quá trình từ VAD đến HAD
sphl = InvVAD(cl, M)
spl = InvHAD(sphl, M)


In [ ]:
# def merge_images(input_folder):
#     images = []
#     max_width = 0
#     total_height = 0
    
#     for filename in os.listdir(input_folder):
#         if filename.endswith(".png"):
#             image_path = os.path.join(input_folder, filename)
#             image = Image.open(image_path)
#             images.append(image)
#             # Cập nhật kích thước tối đa chiều rộng và tổng chiều cao
#             max_width = max(max_width, image.width)
#             total_height += image.height

#     if len(images) == 0:
#         print("Không tìm thấy ảnh trong thư mục")
#         return None

#     # Tạo ảnh mới để lưu trữ ảnh ghép với kích thước tối đa của chiều rộng và tổng chiều cao
#     merged_image = Image.new('RGB', (max_width, total_height))

#     # Vị trí bắt đầu cho việc dán ảnh vào ảnh ghép
#     y_offset = 0

#     # Ghép từng ảnh vào ảnh mới
#     for img in images:
#         merged_image.paste(img, (0, y_offset))
#         y_offset += img.height

#     return merged_image

# if __name__ == "__main__":
#     input_folder = "HAD_VAD"  # Thư mục chứa các ảnh đã khuếch tán
#     merged_image = merge_images(input_folder)
#     if merged_image:
#         merged_image.show()


In [ ]:
ud = int(Kd[:5], 2)
vd = int(Kd[5:10], 2)
rid = int(Kd[10:15], 2)
rjd = int(Kd[15:20], 2)
print(ud,vd,rid,rjd)


In [ ]:
import cv2
import numpy as np

# Đọc hình ảnh
image = cv2.imread('plain_images/lenna512x512.png')

# Chia thành 3 mặt phẳng màu R, G, B
b, g, r = cv2.split(image)

# Tạo các mặt phẳng màu từ các kênh màu
blue_plane = cv2.merge((b, np.zeros_like(g), np.zeros_like(r)))
green_plane = cv2.merge((np.zeros_like(b), g, np.zeros_like(r)))
red_plane = cv2.merge((np.zeros_like(b), np.zeros_like(g), r))

# Hiển thị từng mặt phẳng màu
cv2.imshow('Blue Plane', blue_plane)
cv2.imshow('Green Plane', green_plane)
cv2.imshow('Red Plane', red_plane)

# # Chờ người dùng nhấn một phím bất kỳ để đóng cửa sổ
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# Hàm mã hóa khối 32x32 pixels

# Chia mỗi mặt phẳng màu thành các khối 32x32 pixels
blocks_b = [b[i:i+32, j:j+32] for i in range(0, b.shape[0], 32) for j in range(0, b.shape[1], 32)]
blocks_g = [g[i:i+32, j:j+32] for i in range(0, g.shape[0], 32) for j in range(0, g.shape[1], 32)]
blocks_r = [r[i:i+32, j:j+32] for i in range(0, r.shape[0], 32) for j in range(0, r.shape[1], 32)]
def encrypt_block(block):
    for i in range(32):
        for j in range(32):
            block[i, j] = AES_Sbox[sbox[block[i, j]]]
    return block
def modified_2d_cat_map(i, j, ud, vd, rid, rjd, M):
    Z1 = (rid + rjd)
    Z2 = (ud * vd + 1) 
    Z3 = (vd * i + rjd) 
    in_ = (i + ud * j + Z1) % M
    jn = (Z3 + Z2 * j) % M 
    return in_, jn

# Hàm hoán vị vị trí của các pixel trong một khối
def permute_block(block, Kd):
    ud = int(Kd[:5], 2)
    vd = int(Kd[5:10], 2)
    rid = int(Kd[10:15], 2)
    rjd = int(Kd[15:20], 2)
    M = 32
    for i in range(M):
        for j in range(M):
            in_, jn = modified_2d_cat_map(i, j, ud, vd, rid, rjd, M)
            block[in_, jn] = block[i, j]  # Ghi đè giá trị của pixel vào vị trí mới
    return block
M=32
def HAD(block, M):
    # Tạo một ma trận kích thước MxM để lưu kết quả
    sph = [[0 for _ in range(M)] for _ in range(M)]
    
    # Tính toán giá trị cho từng phần tử trong ma trận
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:  # Trường hợp khi i = j = 1
                sph[i][j] = block[i][j] + (block[M-1][M-1]) % 256 
            elif 0 < i <= M-1 and j == 0:  # Trường hợp khi 1 < i ≤ M và j = 1
                sph[i][j] = block[i][j] + (block[i-1][M-1]) % 256
            elif 0 <= i <= M-1 and 0 < j <= M-1:  # Trường hợp chung
                sph[i][j] = block[i][j] + (block[i][j-1]) % 256
        
    return sph
def VAD(sph, M):
    # Tạo một ma trận kết quả mới kích thước MxM
    cl = [[0 for _ in range(M)] for _ in range(M)]

    # Tính toán giá trị cho từng phần tử trong ma trận cl
    for i in range(M):
        for j in range(M):
            if i == 0 and j == 0:  # Trường hợp khi i = j = 1
                cl[i][j] = sph[i][j] + (sph[M-1][M-1]) % 256 
            elif 0 < j <= M-1 and i == 0:  # Trường hợp khi 1 < i ≤ M và j = 1
                cl[i][j] = sph[i][j] + (sph[i-1][M-1]) % 256
            elif 0 <= j <= M-1 and 0 < i <= M-1:  # Trường hợp chung
                cl[i][j] = sph[i][j] + (sph[i][j-1]) % 256
    
    return cl
# Mã hóa từng khối của các lớp màu RGB
encrypted_blocks_b = [encrypt_block(block) for block in blocks_b]
encrypted_blocks_g = [encrypt_block(block) for block in blocks_g]
encrypted_blocks_r = [encrypt_block(block) for block in blocks_r]

# Hoán vị vị trí của các pixel trong mỗi khối
permuted_blocks_b = [permute_block(block, Kd) for block in encrypted_blocks_b]
permuted_blocks_g = [permute_block(block, Kd) for block in encrypted_blocks_g]
permuted_blocks_r = [permute_block(block, Kd) for block in encrypted_blocks_r]

# Áp dụng HAD và VAD cho mỗi khối đã hoán vị
had_permuted_blocks_b = [HAD(block, M) for block in permuted_blocks_b]
had_permuted_blocks_g = [HAD(block, M) for block in permuted_blocks_g]
had_permuted_blocks_r = [HAD(block, M) for block in permuted_blocks_r]

vad_had_permuted_blocks_b = [VAD(block, M) for block in had_permuted_blocks_b]
vad_had_permuted_blocks_g = [VAD(block, M) for block in had_permuted_blocks_g]
vad_had_permuted_blocks_r = [VAD(block, M) for block in had_permuted_blocks_r]
import cv2

# Ghép các khối đã xử lý thành các mặt phẳng màu
result_b = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(vad_had_permuted_blocks_b, len(blocks_b)//(image.shape[1]//32))])
result_g = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(vad_had_permuted_blocks_g, len(blocks_g)//(image.shape[1]//32))])
result_r = np.vstack([np.hstack(row_blocks) for row_blocks in np.split(vad_had_permuted_blocks_r, len(blocks_r)//(image.shape[1]//32))])

# Ghép lại hình ảnh từ các mặt phẳng màu đã xử lý
result_image = cv2.merge((result_b, result_g, result_r))

# Hiển thị hình ảnh kết quả
cv2.imshow('Result Image', result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
